### Experimenting with FS Ndzomga chess v GPT-4 code



- This is a practice project for me to see if I can:
    1) get this code running on my system which will include learning about the openai API and also some chess tools
    2) find ways to work with the API to do something else.

In [ ]:
import openai
import chess
import chess.engine
import pandas as pd
from mytoken import apikey
import json
from ast import literal_eval

# Initialize the OpenAI API for GPT-4
openai.api_key = apikey

# Initialize a chess board and the chess engine
engine = chess.engine.SimpleEngine.popen_uci("/opt/homebrew/Cellar/stockfish/16/bin/stockfish")

# Read CSV data
df = pd.read_csv('games.csv')

# Convert moves from string to list
df['moves'] = df['moves'].apply(literal_eval)


def is_sublist(lst1, lst2):
    """Check if lst1 is a sublist of lst2"""
    len1, len2 = len(lst1), len(lst2)
    return any((lst1 == lst2[i:len1+i]) for i in range(len2 - len1 + 1))


def find_games(moves_sequence):
    # Check if the sequence exists in the moves
    df['sequence_exists'] = df['moves'].apply(lambda x: is_sublist(moves_sequence, x))

    # Filter rows where sequence_exists is True
    filtered_df = df[df['sequence_exists'] == True]

    # If more than three games exist, take only the first three
    if filtered_df.shape[0] > 3:
        filtered_df = filtered_df.iloc[:3, :]

    # Create a list of dictionaries (each representing a game) for the filtered rows
    # Select only 'moves' and 'winner' columns
    games = filtered_df[['moves', 'winner']].to_dict('records')

    return games


def play_game():
    moves = []  # Variable to store the list of moves
    board = chess.Board()

    def get_gpt4_move(board):
        feedback = ""
        while True:
            similar_games = []
            if len(moves) >= 6:
                for i in range(6, 2, -1):  # Iterate backwards from 6 to 3
                    similar_games = find_games(moves[-i:])
                    if similar_games:
                        break
            prompt = f"Current board: {board}\nMove history: {moves}\nSimilar Games: {similar_games}\n\nChoose the next move for black in UCI format. The available legal moves are {list(board.legal_moves)}. {feedback}"

            response = openai.ChatCompletion.create(
                model="gpt-4-0613",
                messages=[
                    {
                        "role": "system",
                        "content": "You're an AI chess grandmaster. Here's the current game state. Make the best move considering all strategic aspects.",
                    },
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                functions=[
                    {
                        "name": "get_move",
                        "description": "Get the next move for black.",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "next_move": {
                                    "type": "string",
                                    "description": f"The next chess move for black in UCI format. Should be among legal moves: {', '.join(str(move) for move in board.legal_moves)}",
                                }
                            },
                            "required": ["next_move"],
                        },
                    }
                ],
                function_call={"name": "get_move"},
            )

            message = response["choices"][0]["message"]
            if message.get("function_call"):
                arguments = json.loads(message["function_call"]["arguments"])
                next_move = arguments["next_move"]
            try:
                move = chess.Move.from_uci(next_move)
                if move in board.legal_moves:
                    return move
                else:
                    feedback = f"GPT-4's generated move {move} is not valid currently."
            except:
                feedback = "Failed to parse GPT-4's generated move. Retrying..."

    while not board.is_game_over():
        if board.turn:  # True for white's turn, False for black's turn
            result = engine.play(board, chess.engine.Limit(time=2.0))
            board.push(result.move)
            moves.append(result.move.uci())  # Store UCI move in the list
        else:
            move = get_gpt4_move(board)
            board.push(move)
            moves.append(move.uci())  # Store UCI move in the list
        print(board)
        print("\n\n")

    # Check the result of the game
    winner = ""
    if board.is_checkmate():
        if board.turn:
            winner = "Black"
        else:
            winner = "White"
    elif board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition() or board.is_variant_draw():
        winner = "Draw"

    # Create a DataFrame with the game data
    game_data = pd.DataFrame(
        data=[[len(df) + 1, "Stockfish", "GPT-4", moves, winner]],  # Game number is the next one after the last
        columns=["game_number", "white", "black", "moves", "winner"]
    )

    # Append the game to the CSV file
    game_data.to_csv('games.csv', mode='a', header=False, index=False)

    # Return the result
    if winner == "GPT-4":
        return "GPT-4 wins by checkmate."
    elif winner == "Stockfish":
        return "Stockfish wins by checkmate."
    else:
        return "The game is a draw."

# Number of games to play
n_games = 10

# Initialize a dictionary to store the results
results = {"GPT-4 wins": 0, "Stockfish wins": 0, "Draw": 0}

# Run the game n times
for i in range(n_games):
    print(f"Starting game {i+1}...")
    result = play_game()
    print(result)

    # Update the results dictionary based on the outcome of the game
    if "GPT-4 wins" in result:
        results["GPT-4 wins"] += 1
    elif "Stockfish wins" in result:
        results["Stockfish wins"] += 1
    else:
        results["Draw"] += 1

    print(f"Game {i+1} finished.\n\n")

# Print the final results
print("Final results after playing", n_games, "games:")
print("GPT-4 won:", results["GPT-4 wins"], "games")
print("Stockfish won:", results["Stockfish wins"], "games")
print("Draw:", results["Draw"], "games")